In [ ]:
!pip install app_store_scraper
!pip install google-play-scraper

In [ ]:
import pandas as pd
import numpy as np
import json
import time
from tqdm import tqdm

# from app_store_scraper import AppStore
# import google_play_scraper as gps

In [ ]:
# appstore

def App_store_extract_reviews(app_name:str, app_id:int): 
  ott = AppStore(country='kr', app_name=app_name, app_id=app_id)
  ott.review()

  ott_reviews = pd.DataFrame(ott.reviews).reset_index(drop=True)
  ott_reviews['ott'] = app_name
  return ott_reviews

def google_store_extract_reviews(app_name:str, app_id:str):
  result = gps.reviews_all(
        app_id,
        sleep_milliseconds=2000, # 대기시간, 2000값으로 다량의 리뷰를 한번에 불러올때 많은 트래픽으로 인한 문제 방지
        lang='ko', # 한국어 리뷰, 영어리뷰는 'en'
        country='us', # 스토어 국가
        sort=gps.Sort.NEWEST, # 최신순으로 정렬
        filter_score_with=None) # 평점 필터링. None = 전체리뷰평점

  ott_reviews = pd.DataFrame(result)
  ott_reviews['ott'] = app_name
  return ott_reviews

APP_STORE 데이터 수집 및 저장

In [ ]:
ott_platform = {'tiving': (400101401, 'net.cj.cjhv.gs.tving'), 'netflix': (363590051, 'com.netflix.mediaclient'),
                'wave': (987782077, 'kr.co.captv.pooqV2'), 'coupang': (1536885649, 'com.coupang.mobile.play'), 
                'disney': (1446075923, 'com.disney.disneyplus')}

# app_store_review
app_store_reviews = pd.DataFrame()
for i in tqdm(ott_platform):
  ott_reviews = App_store_extract_reviews(i, ott_platform[i][0])
  app_store_reviews = pd.concat([app_store_reviews, ott_reviews], axis=0)
  break
print('App store Done..')


  0%|          | 0/1 [01:05<?, ?it/s]

App store Done..


In [ ]:
app_store_reviews.sort_values('date')
app_store_reviews.sort_values('date').to_csv('app_store_tiving.csv', encoding='utf8')

GOOGLE_STORE 데이터 수집 및 저장

In [ ]:
ott_platform = {'tiving': (400101401, 'net.cj.cjhv.gs.tving'), 'netflix': (363590051, 'com.netflix.mediaclient'),
                'wave': (987782077, 'kr.co.captv.pooqV2'), 'coupang': (1536885649, 'com.coupang.mobile.play'), 
                'disney': (1446075923, 'com.disney.disneyplus')}

# Google_store_review
google_store_reviews = pd.DataFrame()
for i in tqdm(ott_platform):
  ott_reviews = google_store_extract_reviews(i, ott_platform[i][1])
  google_store_reviews= pd.concat([google_store_reviews, ott_reviews], axis=0)

print('Google store Done..')

100%|██████████| 5/5 [20:43<00:00, 248.76s/it]

Google store Done..


In [ ]:
google_store_reviews

,content,at,score,ott
0,검색이 불편해요 타 앱에 비해 제목의 일부만 입력해도 나오는게 없고 거의 제목을 다...,2023-06-04 12:13:00,2,tiving
1,태블릿에 있음,2023-06-04 05:30:28,5,tiving
2,태블릿에 있음,2023-06-04 05:30:16,5,tiving
3,아니 앱에서 쿠폰 코드 등록을 쉽게 해야 앱 쓰죠. 어디다 등록하란건지.,2023-06-04 05:18:00,1,tiving
4,볼게부족해요,2023-06-03 17:52:13,1,tiving
...,...,...,...,...
5582,아직은 볼게 많지 않네요.,2019-12-05 00:26:13,1,disney
5583,infinite loading,2019-11-19 22:13:44,1,disney
5584,Not available in my area Dallas,2019-11-14 15:00:15,1,disney
5585,"I logged in disney plus, but I CAN'T join!!",2019-11-14 14:44:27,2,disney


In [ ]:
google_store_reviews = google_store_reviews[['content', 'at', 'score', 'ott']]
google_store_reviews['at'] = google_store_reviews['at'].map(lambda x: str(x)[:10])
google_store_reviews = google_store_reviews.reset_index(drop=True)
google_store_reviews.to_csv('google_store_reviews.csv', encoding='utf8', escapechar='\\')

-----

데이터 병합

In [ ]:
# 앱스토어 병합
app_store_reviews = pd.DataFrame()
OTT_CONTENTS = ['tiving', 'netflix', 'disney', 'coupang', 'wave']
for ott in tqdm(OTT_CONTENTS):
  review = pd.read_csv(f'app_store_{ott}.csv')

  app_store_reviews = pd.concat([app_store_reviews, review], axis=0)

app_store_reviews = app_store_reviews[['review', 'date', 'rating', 'ott']]
app_store_reviews['date'] = app_store_reviews['date'].map(lambda x: str(x[:10]))
app_store_reviews.columns = ['content', 'at', 'score', 'ott']
app_store_reviews = app_store_reviews.reset_index(drop=True)

app_store_reviews.to_csv('app_store_reviews.csv', encoding='utf8') # 저장

In [ ]:
# 구글스토어 & 앱스토어 병합
app_store_reviews = pd.read_csv('app_store_reviews.csv', encoding='utf8') # 앱스토어
google_store_reviews = pd.read_csv('google_store_reviews.csv', encoding='utf8') # 구글스토어

OTT_CONTENTS = ['tiving', 'netflix', 'disney', 'coupang', 'wave']
review_data = pd.DataFrame()
for ott in OTT_CONTENTS:
  app_ott = app_store_reviews[app_store_reviews['ott'] == ott]
  google_store = google_store_reviews[google_store_reviews['ott'] == ott]

  # (print) 각 스토어에 대한 리뷰데이터 개수
  print(f'ott: {ott} | app_store: {len(app_ott)} | google_store: {len(google_store)}')

  # 각 OTT별 병합
  df_temp = pd.concat([app_ott, google_store], axis=0)
  review_data = pd.concat([review_data, df_temp], axis=0)

ott: tiving | app_store: 3220 | google_store: 21978
ott: netflix | app_store: 2000 | google_store: 25680
ott: disney | app_store: 1408 | google_store: 5587
ott: coupang | app_store: 2000 | google_store: 9280
ott: wave | app_store: 2000 | google_store: 39960


In [ ]:
review_data.iloc[:, 1:].to_csv('review_data.csv', encoding='utf8')

In [ ]:
print('OTT별 보유 리뷰 수')
print('-'*20)
for ott in review_data['ott'].unique():
  ott_temp = review_data[review_data['ott'] == ott]
  print(f'{ott}: {len(ott_temp)} rows')

for ott in review_data['ott'].unique():
  ott_df = review_data[review_data['ott'] == ott]
  ott_df['at'] = pd.to_datetime(ott_df['at'])
  ott_df['year'] = ott_df['at'].dt.year
  print(f'***** {ott} *****')
  for y in sorted(ott_df['year'].unique()):
    tmp = ott_df[ott_df['year'] == y]
    print(f'{y}: {len(tmp)} rows')